In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import mxnet as mx
from gluonts.dataset.pandas import PandasDataset
from gluonts.mx import DeepAREstimator, Trainer, MQRNNEstimator, TemporalFusionTransformerEstimator
# from gluonts.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.trainer.callback import TrainingHistory

from gluonts.dataset.util import to_pandas
from gluonts.dataset.pandas import PandasDataset
from gluonts.evaluation.backtest import make_evaluation_predictions

In [2]:
mx.context.num_gpus()

0

In [3]:
# df = pd.read_csv('Merged-update_hourly.csv')
df = pd.read_csv('../../../data/TFT_energy.csv')

In [4]:
df

,time,price,price_dayahead,gen_coal,gen_gas,load_actual,gen_lig,gen_oil,gen_oth_renew,pressure_Barcelona,...,wind_deg_Bilbao,clouds_all_Bilbao,gen_hyd_river,wind_deg_Seville,wind_speed_Barcelona,wind_speed_Valencia,wind_speed_Bilbao,gen_wind,wind_speed_Madrid,gen_hyd_pump
0,2015-01-01 00:00:00+00:00,64.92,48.10,4755.0,5196.0,24382.0,328.0,158.0,71.0,1035.0,...,229.0,0.0,1009.0,21.0,7.0,1.0,0.0,5890.0,1.0,920.0
1,2015-01-01 01:00:00+00:00,64.48,47.33,4581.0,4857.0,22734.0,323.0,157.0,73.0,1036.0,...,224.0,0.0,973.0,27.0,7.0,0.0,1.0,5461.0,1.0,1164.0
2,2015-01-01 02:00:00+00:00,59.32,42.27,4131.0,4314.0,21286.0,254.0,160.0,75.0,1036.0,...,225.0,0.0,949.0,27.0,7.0,0.0,1.0,5238.0,1.0,1503.0
3,2015-01-01 03:00:00+00:00,56.04,38.41,3840.0,4130.0,20264.0,187.0,156.0,74.0,1036.0,...,221.0,0.0,953.0,27.0,7.0,0.0,1.0,4935.0,1.0,1826.0
4,2015-01-01 04:00:00+00:00,53.63,35.72,3590.0,4038.0,19905.0,178.0,156.0,74.0,1037.0,...,224.0,0.0,952.0,57.0,5.0,2.0,1.0,4618.0,0.0,2109.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35058,2018-12-31 18:00:00+00:00,77.02,68.85,2628.0,7634.0,30653.0,0.0,178.0,95.0,1027.0,...,57.0,0.0,1135.0,30.0,1.0,2.0,0.0,3113.0,1.0,1.0
35059,2018-12-31 19:00:00+00:00,76.16,68.40,2566.0,7241.0,29735.0,0.0,174.0,95.0,1027.0,...,0.0,0.0,1172.0,30.0,3.0,1.0,1.0,3288.0,1.0,1.0
35060,2018-12-31 20:00:00+00:00,74.30,66.88,2422.0,7025.0,28071.0,0.0,168.0,94.0,1028.0,...,140.0,0.0,1148.0,50.0,4.0,3.0,1.0,3503.0,1.0,50.0
35061,2018-12-31 21:00:00+00:00,69.89,63.93,2293.0,6562.0,25801.0,0.0,163.0,93.0,1028.0,...,120.0,0.0,1128.0,60.0,5.0,2.0,1.0,3586.0,2.0,108.0


In [5]:
df.loc[:, 'timestamp'] = pd.to_datetime(df.loc[:, 'time'], infer_datetime_format=True)

In [6]:
df

,time,price,price_dayahead,gen_coal,gen_gas,load_actual,gen_lig,gen_oil,gen_oth_renew,pressure_Barcelona,...,clouds_all_Bilbao,gen_hyd_river,wind_deg_Seville,wind_speed_Barcelona,wind_speed_Valencia,wind_speed_Bilbao,gen_wind,wind_speed_Madrid,gen_hyd_pump,timestamp
0,2015-01-01 00:00:00+00:00,64.92,48.10,4755.0,5196.0,24382.0,328.0,158.0,71.0,1035.0,...,0.0,1009.0,21.0,7.0,1.0,0.0,5890.0,1.0,920.0,2015-01-01 00:00:00+00:00
1,2015-01-01 01:00:00+00:00,64.48,47.33,4581.0,4857.0,22734.0,323.0,157.0,73.0,1036.0,...,0.0,973.0,27.0,7.0,0.0,1.0,5461.0,1.0,1164.0,2015-01-01 01:00:00+00:00
2,2015-01-01 02:00:00+00:00,59.32,42.27,4131.0,4314.0,21286.0,254.0,160.0,75.0,1036.0,...,0.0,949.0,27.0,7.0,0.0,1.0,5238.0,1.0,1503.0,2015-01-01 02:00:00+00:00
3,2015-01-01 03:00:00+00:00,56.04,38.41,3840.0,4130.0,20264.0,187.0,156.0,74.0,1036.0,...,0.0,953.0,27.0,7.0,0.0,1.0,4935.0,1.0,1826.0,2015-01-01 03:00:00+00:00
4,2015-01-01 04:00:00+00:00,53.63,35.72,3590.0,4038.0,19905.0,178.0,156.0,74.0,1037.0,...,0.0,952.0,57.0,5.0,2.0,1.0,4618.0,0.0,2109.0,2015-01-01 04:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35058,2018-12-31 18:00:00+00:00,77.02,68.85,2628.0,7634.0,30653.0,0.0,178.0,95.0,1027.0,...,0.0,1135.0,30.0,1.0,2.0,0.0,3113.0,1.0,1.0,2018-12-31 18:00:00+00:00
35059,2018-12-31 19:00:00+00:00,76.16,68.40,2566.0,7241.0,29735.0,0.0,174.0,95.0,1027.0,...,0.0,1172.0,30.0,3.0,1.0,1.0,3288.0,1.0,1.0,2018-12-31 19:00:00+00:00
35060,2018-12-31 20:00:00+00:00,74.30,66.88,2422.0,7025.0,28071.0,0.0,168.0,94.0,1028.0,...,0.0,1148.0,50.0,4.0,3.0,1.0,3503.0,1.0,50.0,2018-12-31 20:00:00+00:00
35061,2018-12-31 21:00:00+00:00,69.89,63.93,2293.0,6562.0,25801.0,0.0,163.0,93.0,1028.0,...,0.0,1128.0,60.0,5.0,2.0,1.0,3586.0,2.0,108.0,2018-12-31 21:00:00+00:00


In [7]:
df.columns

Index(['time', 'price', 'price_dayahead', 'gen_coal', 'gen_gas', 'load_actual',
       'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona',
       'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia',
       'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville',
       'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river',
       'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia',
       'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump',
       'timestamp'],
      dtype='object')

In [8]:
train_df = df.iloc[:int(len(df)*0.8), :]
test_df = df.iloc[int(len(df)*0.8):, :]

In [9]:
train_df.shape

(28050, 28)

In [10]:
test_df

,time,price,price_dayahead,gen_coal,gen_gas,load_actual,gen_lig,gen_oil,gen_oth_renew,pressure_Barcelona,...,clouds_all_Bilbao,gen_hyd_river,wind_deg_Seville,wind_speed_Barcelona,wind_speed_Valencia,wind_speed_Bilbao,gen_wind,wind_speed_Madrid,gen_hyd_pump,timestamp
28050,2018-03-14 18:00:00+00:00,50.95,37.61,1068.0,5319.0,35044.0,0.0,345.0,102.0,1006.0,...,40.0,1849.0,220.0,1.0,1.0,8.0,14265.0,6.0,761.0,2018-03-14 18:00:00+00:00
28051,2018-03-14 19:00:00+00:00,59.73,48.21,1126.0,5292.0,36230.0,0.0,351.0,101.0,1005.0,...,40.0,1821.0,220.0,4.0,2.0,8.0,13306.0,5.0,149.0,2018-03-14 19:00:00+00:00
28052,2018-03-14 20:00:00+00:00,50.89,41.11,1054.0,4318.0,35065.0,0.0,351.0,103.0,1005.0,...,40.0,1853.0,220.0,1.0,2.0,5.0,12970.0,7.0,168.0,2018-03-14 20:00:00+00:00
28053,2018-03-14 21:00:00+00:00,45.94,36.72,963.0,4047.0,32328.0,0.0,350.0,104.0,1004.0,...,20.0,1853.0,210.0,1.0,1.0,1.0,13090.0,3.0,811.0,2018-03-14 21:00:00+00:00
28054,2018-03-14 22:00:00+00:00,30.59,21.20,853.0,3726.0,29528.0,0.0,329.0,95.0,1004.0,...,0.0,1826.0,220.0,0.0,8.0,5.0,13408.0,6.0,2007.0,2018-03-14 22:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35058,2018-12-31 18:00:00+00:00,77.02,68.85,2628.0,7634.0,30653.0,0.0,178.0,95.0,1027.0,...,0.0,1135.0,30.0,1.0,2.0,0.0,3113.0,1.0,1.0,2018-12-31 18:00:00+00:00
35059,2018-12-31 19:00:00+00:00,76.16,68.40,2566.0,7241.0,29735.0,0.0,174.0,95.0,1027.0,...,0.0,1172.0,30.0,3.0,1.0,1.0,3288.0,1.0,1.0,2018-12-31 19:00:00+00:00
35060,2018-12-31 20:00:00+00:00,74.30,66.88,2422.0,7025.0,28071.0,0.0,168.0,94.0,1028.0,...,0.0,1148.0,50.0,4.0,3.0,1.0,3503.0,1.0,50.0,2018-12-31 20:00:00+00:00
35061,2018-12-31 21:00:00+00:00,69.89,63.93,2293.0,6562.0,25801.0,0.0,163.0,93.0,1028.0,...,0.0,1128.0,60.0,5.0,2.0,1.0,3586.0,2.0,108.0,2018-12-31 21:00:00+00:00


In [12]:
train_ds = PandasDataset(
    train_df,
    target = 'price',
    timestamp = 'timestamp',
    feat_dynamic_real = ['price_dayahead', 'gen_coal', 'gen_gas', 'load_actual',
       'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona',
       'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia',
       'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville',
       'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river',
       'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia',
       'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump'],
    freq = 'H'
)

/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


In [13]:
test_ds = PandasDataset(
    test_df,
    target = 'price',
    timestamp = 'timestamp',
    feat_dynamic_real = ['price_dayahead', 'gen_coal', 'gen_gas', 'load_actual',
       'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona',
       'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia',
       'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville',
       'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river',
       'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia',
       'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump'],
    freq = 'H'
)

/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


#### training

In [14]:
K = 6
n_hours = 24*3

In [15]:
trainer=Trainer(
    learning_rate=1e-3,
    epochs=150,
    num_batches_per_epoch=100,
    add_default_callbacks=True,
    weight_decay=1e-06)

estimator = TemporalFusionTransformerEstimator(
    prediction_length=K,
    freq="H",
#     use_feat_dynamic_real=True,
    batch_size=32,
    trainer=trainer,
    dropout_rate=0.3,
    hidden_dim=64,
    num_heads=2,
)

model = estimator.train(train_ds)

100%|████████████████████████████████████████████████| 100/100 [02:08<00:00,  1.28s/it, epoch=131/150, avg_epoch_loss=0.658]


 13%|██████▎                                          | 13/100 [00:16<01:52,  1.29s/it, epoch=135/150, avg_epoch_loss=0.659]
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/gluonts/mx/trainer/_base.py:474: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(


#### predicted y

In [16]:
quantiles = ['0.5', '0.1', '0.9']
q50 = 0

In [17]:
test_ds_predict_q50_all = np.array([])

for i in range(len(test_df)-n_hours-K+1):
    print(i)
    
    cut = df.iloc[len(train_df)+i:len(train_df)+i+n_hours+K, :]

    test_ds = PandasDataset(
    cut,
    target = 'price',
    timestamp = 'timestamp',
    feat_dynamic_real = ['price_dayahead', 'gen_coal', 'gen_gas', 'load_actual',
       'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona',
       'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia',
       'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville',
       'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river',
       'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia',
       'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump'],
        freq = 'H'
    )
    
    forecast_it, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=model, num_samples=200)
    test_ds_predict = list(forecast_it)
    test_ds_predict_quantiles = test_ds_predict[0].forecast_array
    test_ds_predict_q50 = test_ds_predict_quantiles.mean(axis=0)
    test_ds_predict_q50_all = np.concatenate((test_ds_predict_q50_all, test_ds_predict_q50), axis=None)
    print(test_ds_predict_q50_all)
    print(test_ds_predict_q50_all.shape)
    print('------')

0
[60.95895386 62.3170929  60.35994339 57.06220627 54.21175003 52.3056221 ]
(6,)
------
1
[60.95895386 62.3170929  60.35994339 57.06220627 54.21175003 52.3056221
 65.77928925 63.14543152 59.2564888  56.05879593 54.08368301 48.35015869]
(12,)
------
2
[60.95895386 62.3170929  60.35994339 57.06220627 54.21175003 52.3056221
 65.77928925 63.14543152 59.2564888  56.05879593 54.08368301 48.35015869
 69.05404663 63.22238159 58.39731979 55.55583572 47.99641418 43.27879715]
(18,)
------
3
[60.95895386 62.3170929  60.35994339 57.06220627 54.21175003 52.3056221
 65.77928925 63.14543152 59.2564888  56.05879593 54.08368301 48.35015869
 69.05404663 63.22238159 58.39731979 55.55583572 47.99641418 43.27879715
 61.10197067 56.9473877  53.90563965 47.17899704 43.12962723 40.73991013]
(24,)
------
4
[60.95895386 62.3170929  60.35994339 57.06220627 54.21175003 52.3056221
 65.77928925 63.14543152 59.2564888  56.05879593 54.08368301 48.35015869
 69.05404663 63.22238159 58.39731979 55.55583572 47.99641418 43

/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index repr

[60.95895386 62.3170929  60.35994339 57.06220627 54.21175003 52.3056221
 65.77928925 63.14543152 59.2564888  56.05879593 54.08368301 48.35015869
 69.05404663 63.22238159 58.39731979 55.55583572 47.99641418 43.27879715
 61.10197067 56.9473877  53.90563965 47.17899704 43.12962723 40.73991013
 59.39115906 56.03227615 49.51977539 45.93537903 43.94086838 42.96783447
 55.69194412 49.52316284 45.54243088 43.30282593 41.93707657 41.02626419
 51.46445084 48.58508301 47.32336426 46.60432053 45.81509399 46.06314087
 51.32970047 49.57437134 49.25136948 49.29983521 49.36120605 49.87307358]
(48,)
------
8
[60.95895386 62.3170929  60.35994339 57.06220627 54.21175003 52.3056221
 65.77928925 63.14543152 59.2564888  56.05879593 54.08368301 48.35015869
 69.05404663 63.22238159 58.39731979 55.55583572 47.99641418 43.27879715
 61.10197067 56.9473877  53.90563965 47.17899704 43.12962723 40.73991013
 59.39115906 56.03227615 49.51977539 45.93537903 43.94086838 42.96783447
 55.69194412 49.52316284 45.54243088 

/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index repr

[60.95895386 62.3170929  60.35994339 57.06220627 54.21175003 52.3056221
 65.77928925 63.14543152 59.2564888  56.05879593 54.08368301 48.35015869
 69.05404663 63.22238159 58.39731979 55.55583572 47.99641418 43.27879715
 61.10197067 56.9473877  53.90563965 47.17899704 43.12962723 40.73991013
 59.39115906 56.03227615 49.51977539 45.93537903 43.94086838 42.96783447
 55.69194412 49.52316284 45.54243088 43.30282593 41.93707657 41.02626419
 51.46445084 48.58508301 47.32336426 46.60432053 45.81509399 46.06314087
 51.32970047 49.57437134 49.25136948 49.29983521 49.36120605 49.87307358
 47.32485962 46.71850967 46.62710571 46.63843155 47.20976257 48.85188675
 47.36343384 47.37462997 47.53593063 48.15346909 49.68764877 51.56642151
 46.78956223 47.30720139 48.07258987 49.79286575 51.82439804 53.14533615
 47.73903275 48.45823288 49.90216064 51.60595703 52.67785263 52.8998909
 47.40379715 48.65518188 50.19968796 51.17686462 51.29863358 50.88469315
 46.47934341 47.46457291 48.08561707 48.13472366 47.8

...

/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index repr

[60.95895386 62.3170929  60.35994339 ... 70.58368683 74.63718414
 75.25318146]
(41532,)
------
6922
[60.95895386 62.3170929  60.35994339 ... 75.96602631 77.19631195
 76.52178192]
(41538,)
------
6923
[60.95895386 62.3170929  60.35994339 ... 76.48265076 76.47039795
 75.58489227]
(41544,)
------
6924
[60.95895386 62.3170929  60.35994339 ... 70.77672577 70.40685272
 69.4945755 ]
(41550,)
------
6925
[60.95895386 62.3170929  60.35994339 ... 63.12177658 62.92126465
 62.4275322 ]
(41556,)
------
6926
[60.95895386 62.3170929  60.35994339 ... 67.87561798 66.91182709
 65.25808716]
(41562,)
------
6927
[60.95895386 62.3170929  60.35994339 ... 68.56150055 66.9249649
 66.00881958]
(41568,)
------
6928
[60.95895386 62.3170929  60.35994339 ... 68.10057831 67.67423248
 67.83224487]
(41574,)
------
6929
[60.95895386 62.3170929  60.35994339 ... 67.76793671 68.34588623
 69.76245117]
(41580,)
------
6930
[60.95895386 62.3170929  60.35994339 ... 69.17485809 71.65869141
 76.31973267]
(41586,)
------
6931


/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index repr

[60.95895386 62.3170929  60.35994339 ... 72.49578094 77.33184814
 78.80103302]
(41592,)
------
6932
[60.95895386 62.3170929  60.35994339 ... 76.98318481 77.62355804
 77.08052826]
(41598,)
------
6933
[60.95895386 62.3170929  60.35994339 ... 76.30163574 76.0067215
 74.68325806]
(41604,)
------
6934
[60.95895386 62.3170929  60.35994339 ... 76.20767975 75.55564117
 72.85742188]
(41610,)
------
6935
[60.95895386 62.3170929  60.35994339 ... 76.12711334 72.64134979
 69.78362274]
(41616,)
------


/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


In [18]:
test_ds_predict_q50_supervised = test_ds_predict_q50_all.reshape((-1, K))
test_ds_predict_q50_supervised.shape

(6936, 6)

In [19]:
test_ds_predict_q50_supervised[0:3]

array([[60.95895386, 62.3170929 , 60.35994339, 57.06220627, 54.21175003,
        52.3056221 ],
       [65.77928925, 63.14543152, 59.2564888 , 56.05879593, 54.08368301,
        48.35015869],
       [69.05404663, 63.22238159, 58.39731979, 55.55583572, 47.99641418,
        43.27879715]])

#### actual y

In [20]:
from helper import series_to_supervised, stage_series_to_supervised

In [22]:
test_df_supervised = series_to_supervised(test_df[['price']], n_hours, K)

In [23]:
test_df_supervised.iloc[:, -K:].shape

(6936, 6)

In [24]:
test_df_supervised_y = test_df_supervised.iloc[:, -K:]

In [25]:
test_df_supervised_y

,var1(t),var1(t+1),var1(t+2),var1(t+3),var1(t+4),var1(t+5)
28122,63.57,73.69,67.93,64.33,59.28,55.41
28123,73.69,67.93,64.33,59.28,55.41,53.16
28124,67.93,64.33,59.28,55.41,53.16,49.20
28125,64.33,59.28,55.41,53.16,49.20,48.17
28126,59.28,55.41,53.16,49.20,48.17,46.76
...,...,...,...,...,...,...
35053,71.95,70.85,71.36,75.13,77.61,77.02
35054,70.85,71.36,75.13,77.61,77.02,76.16
35055,71.36,75.13,77.61,77.02,76.16,74.30
35056,75.13,77.61,77.02,76.16,74.30,69.89


#### performance on test set

In [26]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae


print('MAE = {}'.format(float("{:.3f}".format(mae(test_ds_predict_q50_supervised, test_df_supervised_y)))))
print('RMSE = {}'.format(float("{:.3f}".format(np.sqrt(mse(test_ds_predict_q50_supervised, test_df_supervised_y))))))

MAE = 2.778
RMSE = 4.004
